# Videojuegos exitosos

## Índice <a id='back'></a>
* [Introducción](#intro)
* [Etapa 1. Descripción y preprocesamiento de los datos](#data_review)
    * [1.1. Duplicados implícitos](#data_review_duplicated1)
    * [1.2. Valores ausentes](#data_review_missing)
    * [1.3. Llamadas](#data_review_calls)
    * [1.4. Mensajes](#data_review_messages)
    * [1.5. Internet](#data_review_internet)
    * [1.6. Conclusiones](#data_review_conclusion)
* [Etapa 2. Preparación de los datos](#data_prep)
    * [2.1. Tarifa por usuario](#data_prep_income)
* [Etapa 3. Análisis exploratorio de datos](#data_analysis)
    * [3.1. Llamadas](#data_analysis_calls)
    * [3.2. Mensajes](#data_analysis_messages)
    * [3.3. Internet](#data_analysis_mb)
    * [3.4. Ingreso](#data_analysis_income)
    * [3.4. Conclusiones](#data_analysis_conclusions)
* [Etapa 4. Comprobación de hipótesis](#data_hypothesis)
    * [4.1. El ingreso difiere por plan](#data_hypothesis_plan)
    * [4.2. El ingreso difiere por región](#data_hypothesis_zone)
* [Etapa 5. Conclusión general](#data_conclusion)

## Introducción <a id='intro'></a>

Este proyecto es realizado para la tienda online Ice, la cual vende videojuegos por todo el mundo. Se cuenta con datos del año 2016.

**Objetivo e Hipótesis**

El objetivo es identificar patrones para determinar si un juego tiene éxito o no. Se establecen dos hipótesis:
1. Las calificaciones promedio de los usuarios para las plataformas Xbox One y PC son las mismas.
2. Las calificaciones promedio de los usuarios para los géneros de Acción y Deportes son distintas.

**Etapas**
Los datos se almacenan en el archivo `/datasets/games.csv`. No hay ninguna información sobre la calidad de los datos así que se examinarán antes de probar las hipótesis.

Primero, se evaluará la calidad de los datos y si los problemas son significativos, entonces, durante el preprocesamiento de datos, se tomará en cuenta los problemas más críticos. Finalmente se comprobarán las hipótesis para presentar las conclusiones.
 
El proyecto consistirá en cinco etapas:
 1. Descripción y preprocesamiento de los datos.
 2. Preparación de los datos.
 3. Análisis exploratorio de los datos.
 4. Comprobación de hipótesis.
 5. Conclusión general.

[Volver a Contenidos](#back)

## Etapa 1. Descripción y preprocesamiento de los datos <a id='data_review'></a>

Se importan las librerías necesarias.

In [1]:
# Cargar todas las librerías
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats as st
from math import ceil, factorial

Se lee el archivo y se guarda en la variable `df`.

In [2]:
try:
    df = pd.read_csv('games.csv')
except:
    df = pd.read_csv('/datasets/games.csv')

Se imprime la información general y las primeras 10 filas.

In [3]:
df.info()
print()
print(df.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB

                        Name Platform  Year_of_Release         Genre  \
0                 Wii Sports      Wii           2006.0        Sports   
1          Super Mario Bros.      NES           1985.0      Platform   
2             Mario Kart

La tabla cuenta con 16,715 filas y 11 columnas, las cuales son:

1. `'Name'` - Título del videojuego.
2. `'Platform'` - Consola o pltaforma del videojuego.
3. `'Year_of_Release'` - Año de lanzamiento del videojuego.
4. `'Genre'` - Género del videojuego.
5. `'NA_sales'` - ventas en Norteamérica, en millones de dólares estadounidenses.
6. `'EU_sales'` - ventas en Europa, en millones de dólares estadounidenses.
7. `'JP_sales'` - ventas en Japón, en millones de dólares estadounidenses.
8. `'Other_sales'` - ventas en otras regiones, en millones de dólares estadounidenses.
9. `'Critic_Score'` - calificación según la crítica (máximo 100).
10. `'User_Score'` - calificación según los usuarios (máximo 10).
11. `'Rating'` - clasificación ESRB.

Existen valores ausentes en las columnas `'Name'`, `'Year_of_Release'`, `'genre'`, `'Critic_Score'`, `'User_Score'`y `'Rating'`. Se investigará más sobre cada una de ellas. Los nombres de las columnas usan letras mayúsculas y minúsculas, se cambiarán por minúsculas todas las letras.

Se imprimirán los títulos de las columnas en busca de problemas más sútiles.

In [4]:
print(df.columns)

Index(['Name', 'Platform', 'Year_of_Release', 'Genre', 'NA_sales', 'EU_sales',
       'JP_sales', 'Other_sales', 'Critic_Score', 'User_Score', 'Rating'],
      dtype='object')


El único detalle es el uso de mayúsculas y minúsculas. Se cambian a minúsculas todas las letras.

In [5]:
df.columns = df.columns.str.lower()
print(df.columns)

Index(['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales',
       'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating'],
      dtype='object')


A continuación se imprimirán los valores únicos de la columna `'platform'` (la cual no tiene valores ausentes).

In [6]:
print(df['platform'].sort_values().unique())

['2600' '3DO' '3DS' 'DC' 'DS' 'GB' 'GBA' 'GC' 'GEN' 'GG' 'N64' 'NES' 'NG'
 'PC' 'PCFX' 'PS' 'PS2' 'PS3' 'PS4' 'PSP' 'PSV' 'SAT' 'SCD' 'SNES' 'TG16'
 'WS' 'Wii' 'WiiU' 'X360' 'XB' 'XOne']


Estos valores únicos pertenecen a 31 consolas (o plataformas). Se creará un diccionario con estos valores únicos como claves y los nombres completos de las consolas como sus valores (la información fue encontrada en internet). Esto es por si resulta neceario más adelante y para verificar que no existen duplicados implícitos.

In [7]:
platform_dict = {
    '2600': 'Atari 2600',
    '3DO': '3DO Interactive Multiplayer',
    '3DS': 'Nintendo 3DS',
    'DC': 'Sega Dreamcast',
    'DS': 'Nintendo DS',
    'GB': 'Nintendo Game Boy',
    'GBA': 'Nintengo Game Boy Advance',
    'GC': 'Nintendo GameCube',
    'GEN': 'Sega Genesis',
    'GG': 'Sega Game Gear',
    'N64': 'Nintendo 64',
    'NES': 'Nintendo NES',
    'NG': 'Neo Geo',
    'PC': 'Portable Computer',
    'PCFX': 'NEC PC-FX',
    'PS': 'PlayStation',
    'PS1': 'PlayStation 2',
    'PS3': 'PlayStation 3',
    'PS4': 'PlayStation 4',
    'PSP': 'PlayStation Portable',
    'PSV': 'PlayStation Vita',
    'SAT': 'Sega Saturn',
    'SCD': 'Sega CD',
    'SNES': 'Super Nintendo',
    'TG16': 'PC Engine',
    'WS': 'WonderSwan',
    'Wii': 'Nintendo Wii',
    'WiiU': 'Nintendo Wii U',
    'X360': 'Xbox 360',
    'XB': 'Xbox',
    'XOne': 'Xbox One'
}

No existen duplicados implícitos. En la siguiente sección se analizarán los datos de la columna '`name'`.

[Volver a Contenidos](#back)

### 1. 1. Duplicados implícitos <a id='data_review_duplicated1'></a>

Los datos de la columna `'name'` son de tipo object, por lo que pueden existir duplicados implícitos, además contiene dos valores ausentes que pueden deberse a un error en la recopilación de los datos. Lo primero que se realizará es eliminar espacios en blanco al inicio o final, se cambiarán a minúsculas todos los datos y se reemplazarán los dos valores ausentes por `'unknown'`.

In [8]:
df['name'] = df['name'].str.strip() #quita espacios en blanco al inicio o final
df['name'] = df['name'].str.lower()
df['name'] = df['name'].fillna('unknown')

A continuación se imprimirá la cantidad de valores únicos de esta columna.

In [9]:
print(len(df['name'].sort_values().unique()))

11560


Existen 11,560 valores únicos de 16,715 filas, es muy probable que existan duplicados implícitos. Se imprimirán las primeras 10 filas con los títulos en orden.

In [10]:
print(df['name'].sort_values().head(10))

4728                                    '98 koshien
8342                     .hack//g.u. vol.1//rebirth
7087                   .hack//g.u. vol.2//reminisce
8597        .hack//g.u. vol.2//reminisce (jp sales)
8293                  .hack//g.u. vol.3//redemption
1561                        .hack//infection part 1
9063                                    .hack//link
2990                         .hack//mutation part 2
4265                         .hack//outbreak part 3
7992    .hack//quarantine part 4: the final chapter
Name: name, dtype: object


Se oberva que la fila 7087 y 8597 parecen ser el mismo videojuego salvo por `'(jp sales)'`. Se imprimiran las dos filas para investigación.

In [11]:
for number in [7087, 8597]:
    print(df.loc[number])

name               .hack//g.u. vol.2//reminisce
platform                                    PS2
year_of_release                          2006.0
genre                              Role-Playing
na_sales                                   0.11
eu_sales                                   0.09
jp_sales                                    0.0
other_sales                                0.03
critic_score                                NaN
user_score                                  NaN
rating                                      NaN
Name: 7087, dtype: object
name               .hack//g.u. vol.2//reminisce (jp sales)
platform                                               PS2
year_of_release                                     2006.0
genre                                         Role-Playing
na_sales                                               0.0
eu_sales                                               0.0
jp_sales                                              0.16
other_sales                      

Los datos que no son sobre las ventas coinciden y en las ventas la diferencia es que el videojuego etiquetado con `'(jp sales)'` registra únicamente las ventas de Japón. Se filtrará la tabla por filas que contengan la palabra `'sales'` para detectar estos duplicados y eliminarlos.

In [12]:
name_sales = df[df['name'].str.contains('sales')]
print(name_sales['name'].sort_values().unique())

['.hack//g.u. vol.2//reminisce (jp sales)' 'ape escape academy (jp sales)'
 'atelier iris 3: grand phantasm (jp sales)'
 'atelier iris 3: grand phantasm (us sales)'
 'battlefield 2: modern combat(jp sales)' 'bleach: dark souls (jp sales)'
 'bomberman (jp sales)' 'brave story: new traveler (us sales)'
 'brave story: new traveller (jp sales)'
 'chou-kuukan night pro yakyuu king (weekly jp sales)'
 'chou-kuukan night pro yakyuu king(higher jp sales)'
 'dance dance revolution: mario mix (jp sales)'
 'digimon world ds (jp sales)' 'draglade (jp sales)'
 'dragon ball z: budokai tenkaichi 2 (jp sales)'
 'dragon ball z: harukanaru densetsu (jp sales)'
 'dragon ball: origins (jp & incomplete us sales)'
 'dynasty warriors 6 (jp sales)' 'dynasty warriors vol. 2 (jp sales)'
 'earth defense force 2017 (old jp sales)' 'enchanted arms (jp sales)'
 'eternal poison (jp sales)'
 'fatal fury: battle archives volume 1 (jp sales)'
 'fatal fury: battle archives volume 2 (jp sales)'
 'fifa soccer 2003 (weekly

Se oberva que varios títulos contienen entre paréntesis las ventas de alguna o más regiones pero también, existen 3 que contienen más información:
1. `'lunar 2: eternal blue(sales, but wrong system)'`.
2. `'pachi-slot teiou: golgo 13 las vegas (jp sales, but wrong system)'`.
3. `'r-type command (correct us sales)'`.

Se imprimirán las filas que contienen `'lunar 2: eternal blue'` para investigación.

In [13]:
print(df[df['name'].str.contains('lunar 2: eternal blue')])

                                                name platform  \
5331                  lunar 2: eternal blue complete       PS   
9000  lunar 2: eternal blue(sales, but wrong system)      GEN   
9961                           lunar 2: eternal blue      SAT   

      year_of_release         genre  na_sales  eu_sales  jp_sales  \
5331           1999.0  Role-Playing      0.19      0.13      0.00   
9000           1994.0  Role-Playing      0.00      0.00      0.14   
9961           1998.0  Role-Playing      0.00      0.00      0.12   

      other_sales  critic_score user_score rating  
5331         0.02          86.0        8.9      T  
9000         0.00           NaN        NaN    NaN  
9961         0.00           NaN        NaN    NaN  


Son tres filas, con años distintos, esto tienen sentido porque fue lanzado para diferentes consolas. Buscando información, el videojuego del año 1994, fue lanzado para Sega CD (SCD) y no para Sega Genesis (GEN). Se corregirá este dato.

In [14]:
df.loc[9000, 'platform'] = 'SCD'
print(df.loc[9000])

name               lunar 2: eternal blue(sales, but wrong system)
platform                                                      SCD
year_of_release                                            1994.0
genre                                                Role-Playing
na_sales                                                      0.0
eu_sales                                                      0.0
jp_sales                                                     0.14
other_sales                                                   0.0
critic_score                                                  NaN
user_score                                                    NaN
rating                                                        NaN
Name: 9000, dtype: object


Ahora se imprimirán las filas que contienen `'pachi-slot teiou: golgo 13 las vegas'` para investigación.

In [15]:
print(df[df['name'].str.contains('pachi-slot teiou: golgo 13 las vegas')])

                                                    name platform  \
16153  pachi-slot teiou: golgo 13 las vegas (jp sales...      PS2   

       year_of_release genre  na_sales  eu_sales  jp_sales  other_sales  \
16153           2002.0  Misc       0.0       0.0      0.01          0.0   

       critic_score user_score rating  
16153           NaN        NaN    NaN  


Es una única fila y según la información buscada este videojuego fue lanzado para la consola PlayStation. Se modificará este dato.

In [16]:
df.loc[16153, 'platform'] = 'PS'
print(df.loc[16153])

name               pachi-slot teiou: golgo 13 las vegas (jp sales...
platform                                                          PS
year_of_release                                               2002.0
genre                                                           Misc
na_sales                                                         0.0
eu_sales                                                         0.0
jp_sales                                                        0.01
other_sales                                                      0.0
critic_score                                                     NaN
user_score                                                       NaN
rating                                                           NaN
Name: 16153, dtype: object


Se buscarán las filas que contienen `'r-type command'` para información.

In [17]:
print(df[df['name'].str.contains('r-type command')])

                                    name platform  year_of_release     genre  \
11956                     r-type command      PSP           2007.0  Strategy   
13946  r-type command (correct us sales)      PSP           2007.0  Strategy   

       na_sales  eu_sales  jp_sales  other_sales  critic_score user_score  \
11956      0.05       0.0      0.01         0.01          69.0        6.5   
13946      0.04       0.0      0.00         0.00           NaN        NaN   

      rating  
11956   E10+  
13946    NaN  


Únicamente son dos filas, el mensaja entre paréntesis parece indicar que es el dato correcto de las ventas en Norte América. Se corregirá este dato en la primera fila y se eliminará la segunda.

In [18]:
df.loc[11956, 'na_sales'] = 0.04
print(df.loc[11956])

name               r-type command
platform                      PSP
year_of_release            2007.0
genre                    Strategy
na_sales                     0.04
eu_sales                      0.0
jp_sales                     0.01
other_sales                  0.01
critic_score                 69.0
user_score                    6.5
rating                       E10+
Name: 11956, dtype: object


In [19]:
df = df.drop(index=13946).reset_index()
print(df[df['name'].str.contains('r-type command')])

       index            name platform  year_of_release     genre  na_sales  \
11956  11956  r-type command      PSP           2007.0  Strategy      0.04   

       eu_sales  jp_sales  other_sales  critic_score user_score rating  
11956       0.0      0.01         0.01          69.0        6.5   E10+  


Se hará una lista con los mensajes entre paréntesis que se encontraron y que indican las ventas en ciertas regiones para porteriorme quitar los paréntesis de los títulos y eliminar así algunos duplicados ímplicitos.

In [20]:
sales_list = ['(jp sales)', '(us sales)', '(weekly jp sales)', '(higher jp sales)', '(jp & incomplete us sales)',
              '(old jp sales)', '(others sales)', '(american sales)', '(all region sales)', '(jp weekly sales)',
              '(america weekly sales)', '(weekly american sales)', '(us weekly sales)', '(old us sales)',
              '(old all region sales)', '(us & others sales)', '(jp & others sales)', '(japan sales)',
              '(american and others sales)', '(sales, but wrong system)', '(jp sales, but wrong system)', '(all regions sales)']

Se creará una función que recorra los elementos de la lista y evalúe si un título de videojuego contiene alguno de ellos para posteriormente eliminar esta información entre paréntesis.

In [21]:
def name_re(string, sale):
    if sale in string:
        string = string.replace(sale, '')
    else:
        return string
    return string
def name_replace(string):
    for sale in sales_list:
        string = name_re(string, sale)
    return string

Se creará una tabla de prueba y se aplicará la función para verificar que es correcta.

In [22]:
df_test = pd.DataFrame()
df_test['name'] = ['game 1 (jp sales)', 'g2 (us sales)', 'g3(jp weekly sales)', 'g4']
df_test['year'] = [2004, 2005, 2006, 2007]
print(df_test)
print()
df_test['name'] = df_test['name'].apply(name_replace)
print(df_test)

                  name  year
0    game 1 (jp sales)  2004
1        g2 (us sales)  2005
2  g3(jp weekly sales)  2006
3                   g4  2007

      name  year
0  game 1   2004
1      g2   2005
2       g3  2006
3       g4  2007


La función es correcta y además es importante observar que puede dejar espacio en blanco si es que había entre la información antes del paréntesis y el paréntesis. Se aplicará a la tabla, se eliminarán nuevamente los espacios en blanco y se buscarán filas que contengan `'sales'` en el título del dibujo. Si todo es correcto no deben existir estas filas.

In [23]:
df['name'] = df['name'].apply(name_replace)
df['name'] = df['name'].str.strip()
print(df[df['name'].str.contains('sales')])

Empty DataFrame
Columns: [index, name, platform, year_of_release, genre, na_sales, eu_sales, jp_sales, other_sales, critic_score, user_score, rating]
Index: []


Se imprimirá nuevamente la longitud de los valores únicos en la colunmna `'name'`.

In [24]:
print(len(df['name'].sort_values().unique()))

11441


Debido a que es una cantidad muy grande de valores únicos no se pueden revisar uno por uno en busca de ducplicados implícitos (por el tiempo). Se decide tratarlos como 11,441 juegos distintos.

In [33]:
caracter_name = []
for name in df['name']:
    if name[0] not in caracter_name:
        caracter_name.append(name[0])

caracter_name = sorted(caracter_name)
print(caracter_name)

["'", '.', '0', '1', '2', '3', '4', '5', '6', '7', '9', '[', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¡']


In [35]:
caracter_df = {}
for caracter in caracter_name:
    caracter_df[caracter] = df[df['name'].str.startswith(caracter)]

In [40]:
for caracter in caracter_name[:12]:
    print(caracter_df[caracter]['name'].sort_values().unique())

["'98 koshien"]
['.hack//g.u. vol.1//rebirth' '.hack//g.u. vol.2//reminisce'
 '.hack//g.u. vol.3//redemption' '.hack//infection part 1' '.hack//link'
 '.hack//mutation part 2' '.hack//outbreak part 3'
 '.hack//quarantine part 4: the final chapter'
 '.hack: sekai no mukou ni + versus']
['007 racing' '007: quantum of solace' '007: the world is not enough'
 '007: tomorrow never dies']
['1 vs. 100' '1/2 summer +' '10 minute solution' '100 all-time favorites'
 '100 classic books' '100 classic games'
 '1000 cooking recipes from elle à table' '1001 touch games'
 '101-in-1 explosive megamix' '101-in-1 party megamix wii'
 '101-in-1 sports megamix' '101-in-1 sports party megamix'
 '1080°: teneighty snowboarding' '11eyes: crossover'
 '12-sai. honto no kimochi' '12-sai. koisuru diary'
 '12riven: the psi-climinal of integral' '15 days' '1701 a.d.'
 '18 wheeler: american pro trucker' '18 wheels of steel: extreme trucker'
 '18 wheels of steel: extreme trucker 2' '187: ride or die' '1942']
['2 games i

No existen duplicados implícitos.

In [41]:
print(caracter_df['a']['name'].sort_values().unique())

['a boy and his blob' "a bug's life"
 'a collection of activision classic games for the atari 2600'
 'a collection of classic games from the intellivision'
 'a game of thrones: genesis'
 'a good librarian like a good shepherd: library party'
 'a lli s.v.: a ressha de gyoukou 3 super version' 'a new beginning'
 'a ressha de gyoukou 2001' 'a ressha de ikou ds' 'a vampyre story'
 "a witch's tale" 'a-train' 'a-train 6' 'a-train: city simulator'
 'a. iv evolution: a ressha de ikou 4' "a.c.e.: another century's episode"
 "a.c.e.: another century's episode 2"
 "a.c.e.: another century's episode 3: the final"
 "a.c.e.: another century's episode portable"
 "a.c.e.: another century's episode r" 'aa megami-samaa'
 'abba: you can dance' 'abunai: koi no sousa shitsu'
 'ac/dc live: rock band track pack' 'academy of champions: soccer'
 'accel world: ginyoku no kakusei' 'accel world: kasoku no chouten'
 'ace attorney 6' 'ace attorney investigations 2'
 'ace attorney investigations: miles edgeworth'
 '

In [30]:
df_names = df.pivot_table(index='name', values='year_of_release', aggfunc='count')
print(df_names.iloc[50: 100])

                                                    year_of_release
name                                                               
2014 fifa world cup brazil                                        2
24: the game                                                      1
25 to life                                                        2
2xtreme                                                           1
3 in 1: solitaire, mahjong & tangram                              1
300: march to glory                                               1
3d baseball                                                       1
3d dot game heroes                                                1
3d lemmings                                                       1
3d tank warfare simulator panzer front                            1
3ds classic collection                                            1
3rd super robot wars z jigoku hen                                 2
3rd super robot wars z: tengoku-hen             

Se ha notado que algunos títulos contienen información sobre datos de las otras columnas, por esta razón se imprimirán aquellos que contienen paréntesis para una mejor investigación.

In [ ]:
name_extra = df[df['name'].str.endswith(')')]
print(name_extra['name'].sort_values().unique())

Algunos parece que contienen los años de lanzamiento o las plataformas en las que fueron lanzados. Se creará una lista con estos años y una lista con las plataformas.

In [ ]:
name_year = ['(2006)', '(2008)', '(2009)', '(2010)', '(2013)', '(2014)', '(2015)', '(2016)']
name_platforms = ['(ds version)', '(psp)', '(3ds)', '(cd)', '(ps1 & psp versions)', '(console version)', '(ds & mobile versions)',
                  '(ds versions)', '(wii & ps2 version)', '(xbox 360, ps3, & pc versions)',
                  '(xbox 360, ps2, ps3, wii & pc versions)', '(xbox 360, ps3, & pc versions)', '(ds version)', '(gbc)']

Se imprimirán las filas que contienen el año en el títutlo para investigación.

In [ ]:
for year in name_year:
    print(df[(df['name'].str.contains(year)) & (df['name'].str.endswith(')'))])

In [ ]:
print(df[df['name'].str.contains('dragon ball: origins')])

In [ ]:
print(df['platform'].sort_values().unique())

### 1. 1. Valores ausentes <a id='data_review_missing'></a>

Se imprimirá la cantidad de valores ausentes de cada columna para proceder a tratarlos.

In [ ]:
print(df.isna().sum())

Los valores ausentes de las columnas `'name'` y `'genre'` pueden deberse a un error en la recopilación de datos. Como únicamente son dos valores ausentes en cada columna, se reemplazarán por `'unknown'`.

In [ ]:
for column in ['name', 'genre']:
    df[column] = df[column].fillna('unknown')
print(df.isna().sum())

Los valores ausentes en la columna `'year_of_release'` también pueden deberse a un error en la recopilación de los datos. Algunos juegos se lanzan para diversas plataformas o consolas, de manera que se pueden reemplazar estos datos por el año de lanzamiento del juego para otra consola. Para esto, se definirá una función que analice el título del videojuego y asigne el año de lanzamiento para otra consola en caso de existir.

In [ ]:
df['name'] = df['name'].str.strip()
df['name'] = df['name'].str.lower()

In [ ]:
df_year_nan = df[df['year_of_release'].isna()]
print(df_year_nan.head(10))

In [ ]:
df_year_nonan = df.dropna(subset=['name'])
year_nonan = df_year_nonan.pivot_table(index='name', values='year_of_release')
year_nonan.info()
print()
print(year_nonan.head(10))

In [ ]:
def year_release(row):
    name = row['name']
    year = row['year_of_release']
    if pd.isna(year):
        try:
            year = year_nonan.loc[name]
        except:
            year = year
        return year
    else:
        return year

In [ ]:
df_test = pd.DataFrame()
df_test['name'] = ["'98 koshien", '.hack//g.u. vol.1//rebirth', 'Fifa']
df_test['year_of_release'] = [float('nan'), float('nan'), 2014]
print(df_test)
print()
df_test['year_of_release'] = df_test.apply(year_release, axis=1)
print(df_test)

La función es correcta. Se aplicará a `df`.

In [ ]:
df['year_of_release'] = df.apply(year_release, axis=1)
print(df.isna().sum())

In [ ]:
df_year_nan = df[df['year_of_release'].isna()]
print(df_year_nan.head(10))

In [ ]:
print(df[df['name'] == 'rock band'])

In [ ]:
print(df['name'].sort_values().unique())

In [ ]:
print(df[df['name'] == '.hack//g.u. vol.2//reminisce'])
print(df[df['name'] == '.hack//g.u. vol.2//reminisce (jp sales)'])

In [ ]:
name_a = df[df['name'].str.contains('sales')]
print(name_a['name'].sort_values().unique())